# Running Codes with BRAD

# Python Codes

## Seaborn Implementation

## How to add new codes

# Snakemake Pipelines

# MATLAB Codes and Pipelines

This code will inforce:
1. the user must enter the name of a matlab file/script to run that is found within the correct matlab directory

In [77]:
from langchain import PromptTemplate, LLMChain
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
memory = ConversationBufferMemory(ai_prefix="BRAD")

In [67]:
from BRAD import llms
llm = llms.load_nvidia(nvidia_model='meta/llama3-70b-instruct')

Enter your NVIDIA API key:  ········


In [68]:
import os
import glob
import matlab.engine
chatstatus = {
    'prompt' : 'run the eigs command',
    'matlabEng': None,
    'matlab-path':'/home/jpic/RAG-DEV/tutorials/running-codes/matlab/',
    'memory' : ConversationBufferMemory(ai_prefix="BRAD"),
    'llm'    : llm
}

In [69]:
def find_matlab_files(path):

    # Construct the search pattern for .m files
    search_pattern = os.path.join(path, '**', '*.m')
    
    # Find all .m files recursively
    matlab_files = glob.glob(search_pattern, recursive=True)

    # Extract only the file names from the full paths
    file_names = [os.path.basename(file)[:-2] for file in matlab_files]
    
    return file_names

def find_closest_function(query, functions):
    # Find the closest matches to the query in the list of functions
    closest_matches = difflib.get_close_matches(query, functions, n=1, cutoff=0.0)
    
    if closest_matches:
        return closest_matches[0]
    else:
        return None

def read_matlab_docstrings(file_path):
    docstrings = []
    first = True
    with open(file_path, 'r') as file:
        for line in file:
            # print(line)
            stripped_line = line.strip()
            if stripped_line.startswith('%') or first:
                # Remove the '%' character and any leading/trailing whitespace
#                docstring = stripped_line[1:].strip()
                docstrings.append(stripped_line)
                first = False
            else:
                # Stop reading if we encounter a line that doesn't start with '%'
                # assuming docstrings are at the beginning of the file
                break
    return "\n".join(docstrings)


In [73]:
def callMatlab(chatstatus):
    prompt = chatstatus['prompt']                                        # Get the user prompt
    chatstatus['process'] = {}                                           # Begin saving plotting arguments
    chatstatus['process']['name'] = 'Matlab'

    # Add matlab files to the path
    if chatstatus['matlab-path'] == 'matlab-tutorial/':
        matlabPath = chatstatus['matlab-path']
    else:
        base_dir = os.path.expanduser('~')
        matlabPath = os.path.join(base_dir, chatstatus['matlab-path'])
    mpath = eng.addpath(eng.genpath(matlabPath))

    # Identify matlab files we are adding to the path of BRAD
    matlabFunctions = find_matlab_files(matlabPath)

    # Identify which file we should use
    matlabFunction = find_closest_function(prompt, matlabFunctions)

    # Identify matlab arguments
    matlabFunctionPath = os.path.join(matlabPath, matlabFunction + '.m')

    # Get matlab docstrings
    matlabDocStrings = read_matlab_docstrings(matlabFunctionPath)

    # Turn on matlab engine just before running the code
    if chatstatus['matlabEng'] is None:
        chatstatus['matlabEng'] = matlab.engine.start_matlab()
    
    return matlabDocStrings
    
matlabDocStrings = callMatlab(chatstatus)

In [91]:
# Identify the database and the search terms
template = """Current conversation:\n{{history}}

MATLAB FUNCTION DOCUMENTATION:
{matlabDocumentation}

CALL MATLAB FUNCTIONS FROM PYTHON:
Use MATLAB® Engine API for Python® to call any MATLAB function on the MATLAB path. Some examples to call a few common matlab functions include:

To call the matlab function myFnc which has no arguments:
```
eng.myFnc()
```

To add to the matlab path:
```
eng.addpath("C:/work/myfiles")
```

To add a path to all subfolders:
```
s = eng.genpath('C:/work/myfiles')
eng.addpath(s, nargout=0)
```

To use the is prime function which requires an input argument:
```
tf = eng.isprime(37)
```

To determine the greatest common denominator of two numbers, use the gcd function. Set nargout to return the three output arguments from gcd.
```
t = eng.gcd(100.0,80.0,nargout=3)
```

Query:{{input}}

**INSTRUCTIONS**
1. Given the user query and the documentation, identify each of the arguments found in the users query that should be passed to the matlab function.
2. Using the matlab enging eng, provide the one line of code to execute the desired matlab commands. Assume all functions are added to the path and eng already exist.
3. The last line of your response should say "Execute: <Python codes we need to execute>"
"""
chatstatus['prompt'] = 'run my matlabPipeline: NewPipeline with singleCell data file: data.csv'
matlabDocStrings = callMatlab(chatstatus)
filled_template = template.format(matlabDocumentation=matlabDocStrings) #, history=None, input=None)
PROMPT = PromptTemplate(input_variables=["history", "input"], template=filled_template)

conversation = ConversationChain(prompt  = PROMPT,
                                 llm     =    llm,
                                 verbose =   True,
                                 memory  = memory,
                                )

In [88]:
response = conversation.predict(input=chatstatus['prompt'] )
print(response)



> Entering new ConversationChain chain...
Prompt after formatting:
Current conversation:
Human: run my matlab pipeline: NewPipeline with singleCell data file: data.csv
BRAD: Let's break down the user query:

"run my matlab pipeline: NewPipeline with singleCell data file: data.csv"

From the query, we can identify the following arguments:

* `NewPipeline`: This is likely the MATLAB function to be called, equivalent to `AFUN` in the `eigs` documentation.
* `data.csv`: This is the singleCell data file, which will be used as an input to the `NewPipeline` function.

To execute the desired MATLAB command using the `eng` object, we can write:
```
eng.NewPipeline('data.csv')
```
This assumes that `NewPipeline` is a valid MATLAB function that takes a single input argument, which is the file name `data.csv`.
Human: run my matlabPipeline: NewPipeline with singleCell data file: data.csv
BRAD: Let's break down the user query:

"run my matlab pipeline: NewPipeline with singleCell data file: data.c

In [89]:
print(response)

Let's break down the user query:

"run my matlabPipeline: NewPipeline with singleCell data file: data.csv"

From the query, we can identify the following arguments:

* `NewPipeline`: This is the pipelineName argument, which is the name of the pipeline we are running.
* `data.csv`: This is the scPath argument, which is the path to the single cell data file.

To execute the desired MATLAB command using the `eng` object, we can write:
```
eng.matlabPipeline('NewPipeline', 'data.csv')
```
This assumes that `matlabPipeline` is a valid MATLAB function that takes two input arguments, `pipelineName` and `scPath`.


In [92]:
chatstatus['prompt'] = 'run singleCellEnrichment with data.csv'
response = conversation.predict(input=chatstatus['prompt'] )
print(response)



> Entering new ConversationChain chain...
Prompt after formatting:
Current conversation:
Human: run my matlab pipeline: NewPipeline with singleCell data file: data.csv
BRAD: Let's break down the user query:

"run my matlab pipeline: NewPipeline with singleCell data file: data.csv"

From the query, we can identify the following arguments:

* `NewPipeline`: This is likely the MATLAB function to be called, equivalent to `AFUN` in the `eigs` documentation.
* `data.csv`: This is the singleCell data file, which will be used as an input to the `NewPipeline` function.

To execute the desired MATLAB command using the `eng` object, we can write:
```
eng.NewPipeline('data.csv')
```
This assumes that `NewPipeline` is a valid MATLAB function that takes a single input argument, which is the file name `data.csv`.
Human: run my matlabPipeline: NewPipeline with singleCell data file: data.csv
BRAD: Let's break down the user query:

"run my matlab pipeline: NewPipeline with singleCell data file: data.c

In [98]:
eval(response.split('\n')[-1].split(': ')[1])

Error using matlabPipeline
Too many output arguments.



MatlabExecutionError: Too many output arguments.


In [104]:
eval('xx = ' + response.split('\n')[-1].split(': ')[1])

SyntaxError: invalid syntax (<string>, line 1)

In [100]:
eng.matlabPipeline('singleCellEnrichment', 'data.csv')

Argument 1:
singleCellEnrichment
Argument 2:
data.csv

ret = 

    "WORKING!!"



'WORKING!!'

In [74]:
print(matlabDocStrings)

function [V, D, flag] = eigs(varargin)
% EIGS   Find a few eigenvalues and eigenvectors of a matrix
%  D = EIGS(A) returns a vector of A's 6 largest magnitude eigenvalues.
%  A must be square and should be large and sparse.
%
%  [V,D] = EIGS(A) returns a diagonal matrix D of A's 6 largest magnitude
%  eigenvalues and a matrix V whose columns are the corresponding
%  eigenvectors.
%
%  [V,D,FLAG] = EIGS(A) also returns a convergence flag. If FLAG is 0 then
%  all the eigenvalues converged; otherwise not all converged.
%
%  EIGS(A,B) solves the generalized eigenvalue problem A*V == B*V*D. B must
%  be the same size as A. EIGS(A,[],...) indicates the standard eigenvalue
%  problem A*V == V*D.
%
%  EIGS(A,K) and EIGS(A,B,K) return the K largest magnitude eigenvalues.
%
%  EIGS(A,K,SIGMA) and EIGS(A,B,K,SIGMA) return K eigenvalues. If SIGMA is:
%
%      'largestabs' or 'smallestabs' - largest or smallest magnitude
%    'largestreal' or 'smallestreal' - largest or smallest real part
%       

In [39]:
import difflib

# Example query and list of MATLAB functions
query = "ploty"  # The function name specified in the query
matlabFunctions = ["plot", "scatter", "surf", "mesh", "contour", "plot3"]

def find_closest_function(query, functions):
    # Find the closest matches to the query in the list of functions
    closest_matches = difflib.get_close_matches(query, functions, n=1, cutoff=0.0)
    
    if closest_matches:
        return closest_matches[0]
    else:
        return None

# Find the closest MATLAB function
closest_function = find_closest_function(query, matlabFunctions)


In [41]:
help(difflib.get_close_matches)

Help on function get_close_matches in module difflib:

get_close_matches(word, possibilities, n=3, cutoff=0.6)
    Use SequenceMatcher to return list of the best "good enough" matches.
    
    word is a sequence for which close matches are desired (typically a
    string).
    
    possibilities is a list of sequences against which to match word
    (typically a list of strings).
    
    Optional arg n (default 3) is the maximum number of close matches to
    return.  n must be > 0.
    
    Optional arg cutoff (default 0.6) is a float in [0, 1].  Possibilities
    that don't score at least that similar to word are ignored.
    
    The best (no more than n) matches among the possibilities are returned
    in a list, sorted by similarity score, most similar first.
    
    >>> get_close_matches("appel", ["ape", "apple", "peach", "puppy"])
    ['apple', 'ape']
    >>> import keyword as _keyword
    >>> get_close_matches("wheel", _keyword.kwlist)
    ['while']
    >>> get_close_matches

In [1]:
import matlab.engine

In [2]:
eng = matlab.engine.start_matlab()

In [3]:
mpath = eng.addpath(eng.genpath('matlab/'))
eng.myFunc()

Hello Python World from MATLAB!


5.0

In [17]:
eng.helloMATLAB()

Hello World
This message is generated by running MATLAB code from Python.


0.0

In [8]:
eng.myFunc()

Hello Python World from MATLAB!


5.0

In [1]:
memory = ConversationBufferMemory(ai_prefix="BRAD")

# Identify the database and the search terms
template = """Current conversation:\n{{history}}

Available Tables:
{tables}

Query:{{input}}

**INSTRUCTIONS**
1. From the query, decide if GENEONTOLOGY or ENRICHR should be used
2. Identify genes in the users input that should be searched. propose genes with similar names, correct the users spelling, or make small modifications to the list, but do not propose genes that are not found in the humans input.
3. If the user wants to extract genes from a table (python dataframe), provide the necessary code to get the genes into a python list, otherwise, say None.
4. If there is code required, identify the name of the table, otherwise say None
5. If there is code required, identify the row or column name of the table, otherwise say None
6. If there is code required, specify if it is a row or column
Format your output as follows with no additional information:

Database: <ENRICHR or GENEONTOLOGY>
Genes: <List of genes separated by commas in query or None if code is required>
Code: <True or None>
Table: <Table Name>
Mode Name: <Row or Column Name>
Mode: <Row or Column>
"""
tablesInfo = getTablesFormatting(tables)
filled_template = template.format(tables=tablesInfo) #, history=None, input=None)
PROMPT = PromptTemplate(input_variables=["history", "input"], template=filled_template)

conversation = ConversationChain(prompt  = PROMPT,
                                 llm     = llm,
                                 verbose = True,
                                 memory  = memory,
                                )

NameError: name 'ConversationBufferMemory' is not defined